In [17]:
%matplotlib inline
%time from hikyuu.interactive.interactive import *
use_draw_engine('echarts')

Wall time: 0 ns


# 策略分析

## 原始描述

建仓条件：expma周线exp1跟exp2金叉向上使用使用 B=50% 的资金买入股票，建仓成功后，卖出条件才能起作用

卖出条件S1：expma日线exp1和exp2死叉向下时卖出持仓股 S=50%

买入条件B1：expma日线exp1和exp2金叉向上时买入股票数为S（卖出条件S1卖出股数）

S1和B1就这样循环

清仓条件为：expma周线exp1和exp2死叉时


## 策略分析

市场环境：无

系统有效性：周线EMA1（快线）和EMA2（慢线）金叉向上直到两者死叉，系统有效时建立初始仓位

信号指示器：
- 买入：日线EMA1（快线）和EMA2（慢线）金叉向上
- 卖出：日线EMA1（快线）和EMA2（慢线）死叉向下

止损/止盈：无

资金管理：
- 买入：初次建仓时持股数的50%
- 卖出：初次建仓时持股数的50%

盈利目标：无

移滑价差：

In [18]:
def DEMO_CN(self):
    """
    买入信号：周线MACD零轴下方底部金叉，即周线的DIF>DEA金叉时买入
    卖出信号：日线级别 跌破 20日均线
    
    参数：
    week_macd_n1：周线dif窗口
    week_macd_n2: 周线dea窗口
    week_macd_n3: 周线macd平滑窗口
    day_n: 日均线窗口
    """
    k = self.getTO()
    if (len(k) == 0):
        return
    
    #-----------------------------
    #计算周线级别的买入信号        
    #-----------------------------
    #需重新利用日K线的查询条件，构建周线查询条件
    #这里如果用的是hikyuu hdf5的周线数据没问题，如果是其他或自己的数据存储引擎，这里要注意！
    #hikyuu的周线用的是一周的周一生成的Bar，所以QueryByDate使用K[-1].datetime没有问题。
    #而直接使用通达信的周线，则有未来的问题，因为通达信的周线使用的是一周周五的日期
    week_q = QueryByDate(k[0].datetime, k[-1].datetime, kType=Query.WEEK)
    week_k = k.getStock().getKData(week_q)
    
    n1 = self.getParam("week_macd_n1")
    n2 = self.getParam("week_macd_n2")
    n3 = self.getParam("week_macd_n3")
    m = MACD(CLOSE(week_k), n1, n2, n3)
    fast = m.getResult(0)
    slow = m.getResult(1)

    x = fast > slow
    for i in range(x.discard, len(x)):
        if (x[i] >= 1.0):
            self._addValid(week_k[i].datetime)
            #print("buy: ", stk.name, k[i].datetime, len(m), i)

In [19]:
stk = sm['sz000001']
q = Query(-300, kType=Query.DAY) 
sg = crtCN(DEMO_CN, 
              {'week_macd_n1': 12, 'week_macd_n2': 26, 'week_macd_n3': 9, 'day_n': 20}, 
                'DEMO_CN')  
sg.setTO(stk.getKData(q))

In [42]:
#账户参数
init_cash = 100000 #账户初始资金
init_date = '1990-1-1' #账户建立日期

#信号指示器参数
week_n1 = 12
week_n2 = 26
week_n3 = 9
day_n = 20

In [43]:
#创建模拟交易账户进行回测，初始资金30万
my_tm = crtTM(datetime=Datetime(init_date), initCash = init_cash)

#创建系统实例
my_sys = SYS_Simple()

my_sys.tm = my_tm
my_sys.cn = crtCN(DEMO_CN, 
              {'week_macd_n1': 12, 'week_macd_n2': 26, 'week_macd_n3': 9, 'day_n': 20}, 
                'DEMO_CN')  
my_sys.sg = SG_Cross(OP(MA(n=12)), OP(MA(n=26)))
my_sys.mm = MM_FixedCount(1000)

In [47]:
#待测试对象及数据选择
stk = sm['sz000002']
q = Query(-300, kType=Query.DAY) 

my_sys.setParam("cn_open_position", True)
my_sys.run(stk, q)

my_tm.tocsv(sm.tmpdir())

In [48]:
#my_sys.plot()

In [49]:
#绘制资金收益曲线

x = my_tm.getProfitCurve(stk.getDatetimeList(q), KQuery.DAY)
x = PRICELIST(x)
x.plot()